In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Breast_Cancer"
cohort = "GSE283522"

# Input paths
in_trait_dir = "../../input/GEO/Breast_Cancer"
in_cohort_dir = "../../input/GEO/Breast_Cancer/GSE283522"

# Output paths
out_data_file = "../../output/preprocess/Breast_Cancer/GSE283522.csv"
out_gene_data_file = "../../output/preprocess/Breast_Cancer/gene_data/GSE283522.csv"
out_clinical_data_file = "../../output/preprocess/Breast_Cancer/clinical_data/GSE283522.csv"
json_path = "../../output/preprocess/Breast_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# Based on the information in the background metadata, this dataset contains RNA-seq data
# from FFPE breast tumors, indicating it contains gene expression data
is_gene_available = True

# 2.1 Data Availability
# Trait (Breast Cancer) data is available
# Row 6 contains 'sample category' which indicates whether the sample is invasive breast cancer 
# or other types like healthy, DCIS, etc.
trait_row = 6

# Age data is available in row 2
age_row = 2

# Gender/Sex data is available in row 5
gender_row = 5

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait data to binary (0: healthy/no cancer, 1: cancer)."""
    if pd.isna(value):
        return None
    
    # Split by colon and get the value part
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Map sample categories to binary values
    if "invasive breast cancer" in value:
        return 1
    elif "true healthy" in value or "no tumor" in value:
        return 0
    elif "DCIS" in value or "LCIS" in value or "extra ROI" in value:
        # These are precursor lesions or special cases, not considered invasive cancer
        return 0
    elif "positive control" in value:
        # Controls shouldn't be counted as cases
        return None
    else:
        return None

def convert_age(value):
    """Convert age data to continuous values."""
    if pd.isna(value):
        return None
    
    # Split by colon and get the value part
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "not applicable" in value or "missing" in value:
        return None
    
    # Age is given in ranges like "55 - 59"
    if "-" in value:
        # Extract the range and use the midpoint
        try:
            parts = value.replace(' ', '').split('-')
            lower = int(parts[0])
            upper = int(parts[1])
            return (lower + upper) / 2
        except:
            return None
    
    return None

def convert_gender(value):
    """Convert gender/sex data to binary (0: female, 1: male)."""
    if pd.isna(value):
        return None
    
    # Split by colon and get the value part
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "not applicable" in value or "missing" in value:
        return None
    
    if "female" in value.lower():
        return 0
    elif "male" in value.lower():
        return 1
    else:
        return None

# 3. Save Metadata
# Conduct initial filtering on the usability of the dataset
# trait_row is not None, so trait data is available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
# We need to create a DataFrame from the sample characteristics dictionary
# The sample characteristics are the values provided in the previous output

# Create a dictionary to store the sample characteristics for each row
sample_chars = {}
for row_idx, values in Sample_Characteristics_Dictionary.items():
    sample_chars[row_idx] = values

# Convert the dictionary to a DataFrame
clinical_data = pd.DataFrame(sample_chars)

# Now extract the clinical features
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Preview the selected clinical features
preview = preview_df(selected_clinical_df)
print("Preview of clinical features:")
print(preview)

# Save the clinical data to a CSV file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file, index=False)
print(f"Clinical data saved to: {out_clinical_data_file}")


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import os
import pandas as pd
import json
import gzip
import re
from typing import Callable, Optional

# Check what files are available in the cohort directory
cohort_files = os.listdir(in_cohort_dir)
print(f"Files in cohort directory: {cohort_files}")

# Function to parse GEO series matrix file
def parse_geo_series_matrix(file_path):
    with gzip.open(file_path, 'rt') as f:
        lines = f.readlines()
    
    # Extract sample characteristics
    characteristics_rows = {}
    sample_ids = []
    data_start = False
    
    for i, line in enumerate(lines):
        if line.startswith('!Sample_geo_accession'):
            sample_ids = line.strip().split('\t')[1:]
        elif line.startswith('!Sample_characteristics_ch1'):
            parts = line.strip().split('\t')
            header = parts[0]
            values = parts[1:]
            if len(values) > 0:
                row_idx = len(characteristics_rows)
                characteristics_rows[row_idx] = values
        elif line.startswith('!series_matrix_table_begin'):
            data_start = True
            data_start_line = i
            break
    
    # Create clinical dataframe
    clinical_df = pd.DataFrame(index=sample_ids)
    for row_idx, values in characteristics_rows.items():
        clinical_df[f'characteristic_{row_idx}'] = values
    
    # Check if there's gene expression data
    has_gene_data = data_start
    
    return clinical_df, has_gene_data

# Parse the GEO series matrix file
series_matrix_path = os.path.join(in_cohort_dir, "GSE283522_series_matrix.txt.gz")
if os.path.exists(series_matrix_path):
    clinical_data, is_gene_available = parse_geo_series_matrix(series_matrix_path)
    print(f"Clinical data shape: {clinical_data.shape}")
    
    # Display unique values for each sample characteristic
    for i, col in enumerate(clinical_data.columns):
        unique_values = clinical_data[col].unique()
        print(f"Row {i}, Column '{col}': {unique_values}")
else:
    print(f"Series matrix file not found: {series_matrix_path}")
    clinical_data = pd.DataFrame()
    is_gene_available = False

# Identify and process clinical variables
def convert_trait(value):
    if pd.isna(value):
        return None
    
    value = str(value).lower()
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary: 1 for breast cancer, 0 for control/normal
    if any(term in value for term in ['cancer', 'tumor', 'malignant', 'carcinoma']):
        return 1
    elif any(term in value for term in ['normal', 'control', 'benign', 'healthy']):
        return 0
    return None

def convert_age(value):
    if pd.isna(value):
        return None
    
    value = str(value)
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Try to extract age as a number
    age_match = re.search(r'(\d+)', value)
    if age_match:
        return float(age_match.group(1))
    return None

def convert_gender(value):
    if pd.isna(value):
        return None
    
    value = str(value).lower()
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value in ['female', 'f', 'woman']:
        return 0
    elif value in ['male', 'm', 'man']:
        return 1
    return None

# Initialize row indices as None
trait_row = None
age_row = None
gender_row = None

# Look through the sample characteristics to find the appropriate rows
if not clinical_data.empty:
    for i, col in enumerate(clinical_data.columns):
        col_values = clinical_data[col].astype(str).str.lower()
        
        # Check for trait-related information
        trait_terms = ['tissue', 'diagnosis', 'sample type', 'status', 'source', 'histology', 'disease']
        if any(term in col.lower() for term in trait_terms):
            # Check if values indicate cancer/normal distinction
            has_trait_terms = any(('cancer' in val or 'tumor' in val or 'normal' in val or 
                                 'control' in val or 'benign' in val or 'malignant' in val) 
                                for val in col_values)
            # Check if there's more than one unique value
            has_multiple_values = len(set([convert_trait(val) for val in col_values if convert_trait(val) is not None])) > 1
            
            if has_trait_terms and has_multiple_values:
                trait_row = i
        
        # Check for age information
        if 'age' in col.lower():
            # Check if there's more than one unique value after conversion
            ages = [convert_age(val) for val in col_values if convert_age(val) is not None]
            if len(ages) > 1 and len(set(ages)) > 1:
                age_row = i
        
        # Check for gender information
        if 'gender' in col.lower() or 'sex' in col.lower():
            # Check if there's more than one unique value after conversion
            genders = [convert_gender(val) for val in col_values if convert_gender(val) is not None]
            if len(genders) > 1 and len(set(genders)) > 1:
                gender_row = i

# Save metadata - initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path, 
                             is_gene_available=is_gene_available, 
                             is_trait_available=is_trait_available)

# Extract clinical features if trait data is available
if is_trait_available:
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data, 
        trait=trait, 
        trait_row=trait_row, 
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    # Preview the selected clinical features
    preview = preview_df(selected_clinical_df)
    print(f"Preview of selected clinical features: {preview}")
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to: {out_clinical_data_file}")


### Step 4: Gene Data Extraction

In [ ]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
try:
    with gzip.open(matrix_file, 'rt') as file:
        for line in file:
            if "!series_matrix_table_begin" in line:
                found_marker = True
                break
    
    if found_marker:
        print("Found the matrix table marker in the file.")
    else:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        
    # Try to extract gene data from the matrix file
    gene_data = get_genetic_data(matrix_file)
    
    if gene_data.shape[0] == 0:
        print("Warning: Extracted gene data has 0 rows.")
        is_gene_available = False
    else:
        print(f"Gene data shape: {gene_data.shape}")
        # Print the first 20 gene/probe identifiers
        print("First 20 gene/probe identifiers:")
        print(gene_data.index[:20].tolist())
        
except Exception as e:
    print(f"Error extracting gene data: {e}")
    is_gene_available = False
    
    # Try to diagnose the file format
    print("Examining file content to diagnose the issue:")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            for i, line in enumerate(file):
                if i < 10:  # Print first 10 lines to diagnose
                    print(f"Line {i}: {line.strip()[:100]}...")  # Print first 100 chars of each line
                else:
                    break
    except Exception as e2:
        print(f"Error examining file: {e2}")

if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")